## Perform imports and load the dataset

In [55]:
import numpy as np
import pandas as pd
#!pip install nltk
#import nltk
#nltk.download()

In [56]:
import re # for regular expressions
pd.set_option("display.max_colwidth", 200)
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk # for text manipulation
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [57]:
df_train = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/test.csv')

In [58]:
sample_submission = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/sample_submission.csv')

In [59]:
sample_submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,1


In [60]:
df_train.head()

,sentiment,message,tweetid
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, what!? https://t.co/yeLvcEFXkC via @mashable",625221
1,1,It's not like we lack evidence of anthropogenic global warming,126103
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #ElectionNight",466954


In [61]:
df_test.head()

,message,tweetid
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928


In [62]:
print(len(df_train))

15819


In [63]:
print(len(df_test))

10546


In [64]:
df_train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [65]:
df_test.isnull().sum()

message    0
tweetid    0
dtype: int64

In [66]:
df_train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [67]:
# Check for whitespace strings (it's OK if there aren't any!):
blanks = []  # start with an empty list

for i,sntm,msg,twts in df_train.itertuples():  # iterate over the DataFrame
    if type(msg)==str:            # avoid NaN values
        if msg.isspace():         # test 'message' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
len(blanks)

0

In [68]:
df = df_train.copy()

In [69]:
#Split the train data into train and validation 

from sklearn.model_selection import train_test_split

X = df["message"]
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Build a pipeline to vectorize the date, then train and fit a model
Use LinearSVC..

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [71]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [72]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 198   50  157   23]
 [  28  319  319   71]
 [  38  125 2439  259]
 [   4   23  214  954]]


In [73]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.74      0.46      0.57       428
           0       0.62      0.43      0.51       737
           1       0.78      0.85      0.81      2861
           2       0.73      0.80      0.76      1195

    accuracy                           0.75      5221
   macro avg       0.72      0.64      0.66      5221
weighted avg       0.74      0.75      0.74      5221



In [74]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.748898678414097


## Combine Steps with TfidVectorizer

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(10598, 23895)

## Train a Classifier
Here we'll introduce an SVM classifier that's similar to SVC, called LinearSVC. LinearSVC handles sparse input better, and scales well to large numbers of samples.

In [76]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

## Build a Pipeline

In [77]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

## Test the classifier and display results

In [78]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [79]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 198   50  157   23]
 [  28  319  319   71]
 [  38  125 2439  259]
 [   4   23  214  954]]


In [80]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.74      0.46      0.57       428
           0       0.62      0.43      0.51       737
           1       0.78      0.85      0.81      2861
           2       0.73      0.80      0.76      1195

    accuracy                           0.75      5221
   macro avg       0.72      0.64      0.66      5221
weighted avg       0.74      0.75      0.74      5221



In [81]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.748898678414097


## Create csv to upload to Kaggle

In [82]:
#Make predictions using the features from the test data set
predictions = text_clf.predict(df_test['message'])
#predictions = clf.predict(df_test[])

#Display our predictions - they are either 0 or 1 for each training instance 
#depending on whether our algorithm believes the person survived or not.
predictions

array([1, 1, 1, ..., 2, 0, 1], dtype=int64)

In [83]:
#Create a  DataFrame with the tweetid and our prediction regarding whether it good or bad
submission = pd.DataFrame({'tweetid':df_test['tweetid'],'sentiment':predictions})

#Visualize the first 5 rows
submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0


In [84]:
#This is saved in the same directory as your notebook
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission.csv


# Cleaning the data to improve the accuracy results

In [105]:
#copy the training dataset
train = df_train.copy()
test = df_test.copy()

In [86]:
#using regular expressions to remove tagy symbols
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [106]:
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['message'], "@[\w]*") 
test.head()

,message,tweetid,tidy_tweet
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange ;.."
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263,\nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928,RT : 'Female orgasms cause global warming!'\n-Sarcastic Republican


In [87]:
#
train['tidy_tweet'] = np.vectorize(remove_pattern)(train['message'], "@[\w]*") 
train.head()

,sentiment,message,tweetid,tidy_tweet
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, what!? https://t.co/yeLvcEFXkC via @mashable",625221,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, what!? https://t.co/yeLvcEFXkC via"
1,1,It's not like we lack evidence of anthropogenic global warming,126103,It's not like we lack evidence of anthropogenic global warming
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…,698562,RT : Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD,573736,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #ElectionNight",466954,"RT : It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #ElectionNight"


In [107]:
#removing spacial charectors using regular expressions
test['tidy_tweet'] = test['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
test.head()

,message,tweetid,tidy_tweet
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760,Europe will now be looking to China to make sure that it is not alone in fighting climate change https t co O T rCgwDq
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326,Combine this with the polling of staffers re climate change and womens rights and you have a fascist state https t co ifrm eexpj
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985,The scary unimpeachable evidence that climate change is already here https t co yAedqcV Ki #itstimetochange #climatechange
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263,Putin got to you too Jill Trump doesn t believe in climate change at all Thinks it s s hoax
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928,RT Female orgasms cause global warming Sarcastic Republican


In [115]:
#removing spacial charectors using regular expressions
train['tidy_tweet'] = train['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
train.head()

KeyError: 'tidy_tweet'

In [113]:
#remove 3 letter word
test['tidy_tweet'] = test['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test.head()

,message,tweetid,tidy_tweet
0,Europe will now be looking to China to make sure that it is not alone in fighting climate change… https://t.co/O7T8rCgwDq,169760,Europe will looking China make sure that alone fighting climate change https rCgwDq
1,Combine this with the polling of staffers re climate change and womens' rights and you have a fascist state. https://t.co/ifrm7eexpj,35326,Combine this with polling staffers climate change womens rights have fascist state https ifrm eexpj
2,"The scary, unimpeachable evidence that climate change is already here: https://t.co/yAedqcV9Ki #itstimetochange #climatechange @ZEROCO2_;..",224985,scary unimpeachable evidence that climate change already here https yAedqcV #itstimetochange #climatechange
3,@Karoli @morgfair @OsborneInk @dailykos \nPutin got to you too Jill ! \nTrump doesn't believe in climate change at all \nThinks it's s hoax,476263,Putin Jill Trump doesn believe climate change Thinks hoax
4,RT @FakeWillMoore: 'Female orgasms cause global warming!'\n-Sarcastic Republican,872928,Female orgasms cause global warming Sarcastic Republican


In [112]:
#remove 3 letter word
train['tidy_tweet'] = train['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

KeyError: 'tidy_tweet'

In [110]:
#plitting sentaces into words
tokenized_tweet1 = test['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

In [111]:
#plitting sentaces into words
tokenized_tweet = train['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

KeyError: 'tidy_tweet'

In [91]:
tokenized_tweet.head()

0    [PolySciMajor, chief, doesn, think, carbon, dioxide, main, cause, global, warming, wait, what, https, yeLvcEFXkC]
1                                                               [like, lack, evidence, anthropogenic, global, warming]
2                           [Researchers, have, three, years, climate, change, before, late, https, KdUr, https, ANPT]
3                                             [#TodayinMaker#, WIRED, pivotal, year, climate, change, https, wOTxTLcD]
4                                    [racist, sexist, climate, change, denying, bigot, leading, polls, #ElectionNight]
Name: tidy_tweet, dtype: object

In [92]:
# use stemming (gives two meaning words)
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [93]:
#change to sentances again after stemming was complete
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    
train['tidy_tweet'] = tokenized_tweet

In [94]:
#importing word to vector model
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [95]:
tokenized_tweet = train['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

model_w2v = Word2Vec(tokenized_tweet, size=200,  window=5)

model_w2v.train(tokenized_tweet, total_examples= len(train['tidy_tweet']), epochs=20)

(2224556, 3557100)

In [96]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [97]:
#
wordvec_arrays = np.zeros((len(tokenized_tweet), 200))

for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(15819, 200)

In [98]:
from sklearn.model_selection import train_test_split


train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]

_, _, ytrain, yvalid = train_test_split(train_w2v, train['sentiment'],  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, train['sentiment'].shape)

xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

(15819, 200) (15819,)


In [100]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [101]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [102]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 198   50  157   23]
 [  28  319  319   71]
 [  38  125 2439  259]
 [   4   23  214  954]]


In [103]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.74      0.46      0.57       428
           0       0.62      0.43      0.51       737
           1       0.78      0.85      0.81      2861
           2       0.73      0.80      0.76      1195

    accuracy                           0.75      5221
   macro avg       0.72      0.64      0.66      5221
weighted avg       0.74      0.75      0.74      5221



In [104]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.748898678414097


In [99]:
from tensorflow.contrib.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import f1_score

epochs = 25
batch_size = 1024
loss = "binary_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]

from keras import models

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# Build neural network
model = models.Sequential()
model.add(Dense(512, activation='relu', input_shape=(200,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=loss,optimizer=optimizer,metrics= metrics)
model.fit(xtrain_w2v,ytrain,epochs=epochs,batch_size=batch_size,callbacks=callbacks,validation_data=(xvalid_w2v,yvalid))


predictions = model.predict(xvalid_w2v)
predictions = [0 if i<0.5 else 1 for i in predictions]

f1_score(yvalid, predictions)
# print("Accuracy: ",accuracy_score(ytrain,predictions))
# print("Classification Report: ",classification_report(ytrain,predictions))


ModuleNotFoundError: No module named 'tensorflow'